In [3]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import timm
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_score, recall_score
import numpy as np

In [8]:
# Rutas
DATA_PATH = "data/"
ORIGINAL_IMAGE_PATH = os.path.join(DATA_PATH, "images")
SYNTHETIC_IMAGE_PATH = os.path.join(DATA_PATH, "synthetic_images")
TRAIN_CSV_PATH = os.path.join(DATA_PATH, "train_split.csv")
VAL_CSV_PATH = os.path.join(DATA_PATH, "val_split.csv")

In [9]:
print(ORIGINAL_IMAGES_PATH, SYNTHETIC_IMAGE_PATH, TRAIN_CSV_PATH, VAL_CSV_PATH)

data/images data/synthetic_images data/train_split.csv data/val_split.csv


## 2. Definición de transformaciones de imagenes

In [15]:
# Obtener data de entrenamiento modelo preentrenado HuggingFace
MODEL_NAME = 'edgenext_base.in21k_ft_in1k'
model_cfg = timm.get_pretrained_cfg(MODEL_NAME)
IMG_SIZE = model_cfg.input_size[1]
NORM_MEAN = model_cfg.mean
NORM_STD = model_cfg.std

In [16]:


train_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
    transforms.ToTensor(),
    transforms.Normalize(mean=NORM_MEAN, std=NORM_STD),
])

val_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=NORM_MEAN, std=NORM_STD),
])

## 3. Crear DataLoader